# Seruminstituttet og Sundhedsstyrelses prognose af 22. marts 2020.


Denne *notebook* baserer sig på modellen, der er skitseret i notatet,
*Håndtering af COVID-19: Prognose og kapacitet i Danmark for intensiv
terapi* [ITA_COVID_19_](ITA_COVID_19_220320.pdf)

Notatet er udarbejdet sammen med RUC uden nærmere angivelse, måske 
[professor Lone Simonsen](https://forskning.ruc.dk/da/persons/lonesimo), der forsker i modellering 
af pandemier.

Lad os komme i gang. Først har vi brug for nogle databehandlingspakker:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint

## SIR-modellen
[SIR-modellen](https://da.wikipedia.org/wiki/SIR-modellen) er en simpel infektionsmodel.

S, I, og R står for henholdsvis *Susceptible* (hvor man kan få sygdommen, som ikke har haft den endnu), 
*Infectuous* (hvor mange har den nu og kan smitte andre), og
*Recovered* (dem der ikke længere har sygdommen, både de raske og de døde).

Udviklingen over tid beskrives med nogle simple differentialligninger.

Modellen er også vist med kodeeksempler i [SciPython-bogen](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/).
Disse danner udgangspunkt for nedenstående.


## Befolkningsantagelser
**NB** Notatet skifter mellem en befolkning på 5,8 mio. (figur 1 på side 3) 
og 5,7 mio. personer i Danmark.

Vi regner med 5,8, som stemmer med Danmarks Statistiks [Folketal](https://www.dst.dk/da/Statistik/emner/befolkning-og-valg/befolkning-og-befolkningsfremskrivning/folketal).

In [ ]:
# Total befolkning
BEFOLKNINGSTAL = 5_800_000

## Smitteantagelser
$\beta$ (beta) beskriver hvor smitsom sygdommen er.

$\gamma$ (gamma) beskriver hvor hurtigt folk holder op med at være smitsomme, det reciprokke af 
 hvor mange dage de smitter.

*Reproduktionstallet* $R_0$ (engelsk, "R-nought") angiver hvor mange andre en 
smittet person smitter.

I SIR-modellen kan det udtrykkes som:

$$R_0 = {\beta\over\gamma}$$

In [ ]:
# Notatet antager, at folk kan smitte i 7 dage (dvs. 1/7 holder op med at smitte pr dag)
gamma = 1./7

# Notatet antager, at syge smitter 2,6 personer 
# (dette er reproduktionsraten R0, forholdet mellem beta og gamma)
R0 = 2.6
beta = R0*gamma

In [ ]:
# Tid i dage (notatet siger 12 uger, men figur 2 er regnet på 15 uger)
UGER = 15
t = np.linspace(0, UGER*7, UGER*7)

In [ ]:
# SIR modellens differentialligninger
def afledte(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Udgangspunktet, antal personer i hhv. tilstandene, S, I og R
# Vi anvender _start som betegnelse for ikke at skabe
# forvirring om reproduktionstallet R0 og startværdien for R.

def sir_prognose(angrebsrate, befolkningstal):
    assert 0 < angrebsrate < 1.0
    assert befolkningstal > 0
    
    # Det er ikke alle i befolkningen, der er modtagelige
    N = angrebsrate * befolkningstal
    I_start, R_start = 1, 0
    # Resten er *susceptible* ved start, Sstart.
    S_start = N - I_start - R_start
    
    # Starttilstandsvektor
    y_start = S_start, I_start, R_start
    
    # Integrer SIR-modellen equations over the tidsaksen, t.
    ret = odeint(afledte, y_start, t, args=(N, beta, gamma))
    S, I, R = ret.T
    
    return S.astype(np.int), I.astype(np.int), R.astype(np.int)

In [ ]:
S,I,R = sir_prognose(angrebsrate=0.10, befolkningstal=BEFOLKNINGSTAL)

Sådan ser totalerne ud: 

In [ ]:
# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')

ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t, S/1_000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1_000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, R/1_000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Tid (dage)')
ax.set_ylabel('Antal (1000er)')
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='grey', lw=1, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

Lad os prøve at reproducere notatets **figur 2** på side 4.

Her laver de en kurve for Lombardiet, Veneto og Emilia Romagna.
Sidenhen skalerer de så til danske forhold.

Vi skal således beregne kurven for de tre italienske regioner:

In [ ]:
S,I,R = sir_prognose(angrebsrate=0.10, befolkningstal=18_000_000)

In [ ]:
# Nye smittede er dem der falder ud af S (raskhedstilstanden) 
delta_smittede = -np.diff(S, n=1) # deltaerne fra dag 2 og frem
max_nye_smittede = np.max(delta_smittede)
max_I = np.max(I)
t_top = np.argmax(delta_smittede)

print(f'max nye smittede på en dag = {max_nye_smittede}')
print(f'max I = {max_I}')
print(f'dage til top = {t_top} ')

In [ ]:
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(t[1:], delta_smittede, 'r', alpha=0.5, lw=2, label='Nye smittede')
plt.suptitle(f'Figur 2 fra notatet')
plt.grid(True)
plt.axhline(y=max_nye_smittede, color='grey', linestyle='--', label="max daglige smittede")
plt.show()

Dette stemmer overhovedet ikke med den figuren i notatet, der har en top på under 5000 efter ca. 6 uger (41 dage).

Det kan skyldes en kodefejl i indeværende notesbog.

Figuren i notatet viser antal dagligt indberettede data.
Givet at ikke alle tilfælde indberettes, kan der være en konstant faktor, der mangler, idet
vi regner på alle tilfælde.

Det kan dog ikke forklare hele forskellen, da vores model også
giver en senere top (ca. 60 dage). 

Dette kunne tyde på, at de har regnet med en højere $R_0$ og dermed en højere $\beta$
end angivet.

## Åbne spørgsmål

- kontroller vores model mod kendte eksempler for at sikre, at den er implementeret korrekt (f.eks. Hong Kong influenzaen in 1968). 

 
## Empiriske data
Data kan hentes fra Sundhedsstyrelsen, [Tal og overvågning](https://www.sst.dk/da/corona/tal-og-overvaagning).
